<a href="https://colab.research.google.com/github/jamg-upv/genAI4ResearchTeaching/blob/main/asepuc25/chatgptAPI_PROMPT_UBschoolMay25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descripción del caso de uso GPTUS01-entity_extraction
basado en Reto21-chatgptAPI-CasoUso1HIWPclasificacion.ipynb

Clonar el bot que tengo en poe.com sobre entity extraction y clasificación de resumenes de artículos.

LA entrada es una tabla con dos columnas
ID|abstract
El resultado quiero que me genere una tabla con
Id|Entity extraction separada por ";"|clasificacion|rationale para la clasificación

Manualmente copiando y pegando el input y los resultados (de 20 en 20) me funciona dentro de la ventana de contexto. Pero quiero evitar tener que hacerlo a mano.

# Archivo de trabajo

In [1]:
# En Python, puedes usar comillas simples o dobles para definir una cadena de texto (string). Ambas son válidas y no hay diferencia funcional entre ellas.
#  si tu cadena de texto contiene una comilla simple (como en "I'm"), puedes usar comillas dobles para evitar tener que escapar la comilla simple. De manera similar, si tu cadena de texto contiene una comilla doble, puedes usar comillas

# RutaData = input("Ruta completa al archivo (ejemplo: https://gdu.uv.es/sh/alfresco.xls ): ")

RutaData="WOSsavedrecsSET1311results_Short.xlsx"

# Este código lee el archivo XLS ubicado en la URL especificada y lo carga en un DataFrame de pandas llamado DataPapers.
# Debes tener en cuenta que read_excel() necesita la biblioteca openpyxl o xlrd para leer archivos de Excel,
# si no la tienes instalada, puedes hacerlo con pip install openpyxl o pip install xlrd.
# Además, también necesitas tener la biblioteca requests para leer archivos desde una URL. Si no la tienes instalada, puedes hacerlo con pip install requests.

import pandas as pd

# Leemos el archivo en un DataFrame
DataPapers = pd.read_excel(RutaData)

DataPapers['Prompt'] = DataPapers['Id'].astype(str) + "# "  + DataPapers['Title'] + ". "+ DataPapers['Abstract']
DataPapers

# Definimos las columnas que queremos mantener
columnas = ["Id", "Title","Abstract", "Prompt"]

# Seleccionamos las columnas y las primeras 8 filas
# DataPapers = DataPapers[columnas].iloc[:8]

# Seleccionamos las columnas y TODAS las filas (no solo las primeras 10)
DataPapers = DataPapers[columnas]  # Eliminamos el .iloc[:10]


DataPapers



,Id,Title,Abstract,Prompt
0,id0001_0,,Study on Data Analysis of Assessment in Class...,id0001_0# . Study on Data Analysis of Assess...
1,id0012_0,,Flipped pedagogy and student evaluations of t...,id0012_0# . Flipped pedagogy and student eva...
2,id0021_0,,ntegrity of Attitudes and Rating Tendencies o...,id0021_0# . ntegrity of Attitudes and Rating...
3,id0022_2,,Making Meaning from Student Evaluations of Te...,id0022_2# . Making Meaning from Student Eval...
4,id0024_0,,Student evaluations of teaching (SET): Guidel...,id0024_0# . Student evaluations of teaching ...
...,...,...,...,...
189,id0692_1,,How Does Students' Negative Feedback Affect U...,id0692_1# . How Does Students' Negative Feed...
190,id0697_1,,Lexical based automated teaching evaluation v...,id0697_1# . Lexical based automated teaching...
191,id0702_1,,Exploring student engagement in fully flipped...,id0702_1# . Exploring student engagement in ...
192,id0703_1,,TEACHING EVALUATIONS AS A SITE OF INSTITUTION...,id0703_1# . TEACHING EVALUATIONS AS A SITE O...


# Convertir en un bucle y usar funcion para tener un codigo más limpio

In [2]:
PromtSystemB = '''
# Role and Context
You are an [Pedagogy] scientist specializing in screening scientific articles. You will analyze articles based on how they align with this research:
["Research literature reviews that analyse the Impact of Open-ended Student Teaching Evaluations on University Professors' Psychological Health and Well-being"]

# Key Definition and Components
[Student evaluation of teaching open-ended comments qualitative feedback psychological impact faculty mental health wellbeing. Anonymous student feedback negative comments abusive commentary emotional distress stress anxiety depression academic staff. Open-ended questions free-response written comments student evaluations teaching SET formative feedback instructor psychological wellbeing professional confidence. Narrative feedback qualitative data student commentary teaching effectiveness emotional impact academics higher education faculty stress mental health outcomes]

# Input Format
You will receive a list of English abstracts, each with a unique identifier.

# Processing Instructions
For each abstract, follow these four steps:

1. Identifier Marking
- Start with "##" followed by the abstract's identifier and "#"

2. Entity Extraction
- Extract relevant entities from the abstract related to Key Definition and Components
- Separate entities with semicolons (;)
- End with "#"

3. Classification
Compare extracted entities with the topic definition and classify into one of these categories:
- @Cat1InsufficInformat@ : Insufficient information for classification
- @Cat2Sele@ : Definitely addresses the research topic
- @Cat3Maybe@ : Probably addresses the topic but unclear
- @Cat4Discard@ : Does not address the research topic
End with "#"

 4. Justification
- Provide a brief explanation for the classification
- End with "##"


# Classification Criteria
To be classified as @Cat2Sele@, abstract must show clear alignment with "Key Definition and Components"

# Sample Analysis
Input format:
ID: [ABC123]   Abstract: [Abstract text]

Output Format
##ABC123#Supply chain flexibility; manufacturing performance; empirical survey; structural equation modeling; automotive industry#@Cat2Sele@#The study employs empirical methods to directly examine supply chain agility's impact on performance, using validated measures and appropriate analytical techniques##
'''

# #  la "temperatura" es un hiperparámetro que controla la aleatoriedad de las predicciones del modelo.
#  Un valor de temperatura más alto (como 0.8 o 1.0) hará que el modelo haga predicciones más diversas y creativas,
#  mientras que un valor más bajo (como 0.2 o 0.3) hará que las predicciones sean más concentradas y deterministas.
# # Dado que tu tarea implica la extracción de entidades, la clasificación y la explicación,
# querrías que las respuestas del modelo sean coherentes y directas en lugar de ser demasiado creativas.
# Por lo tanto, te recomendaría una temperatura más baja, posiblemente alrededor de 0.3.

Temperatura=0.3 #por encima de 0.5 pocas veces sigue las instrucciones (no llega al cuarto paso o no pone los # donde toca)



In [3]:
# Crear un DataFrame vacío
df3 = pd.DataFrame(columns=['GPTId', 'GPTContent','GPTResponse','GPTUsage','PromtAgregado'])

# Codigo especifico para OPEN AI API




##introducir api key OpenAI (jecutar una vez cada sesión)

In [ ]:
#manual
import getpass

# Solicitar la entrada al usuario
keyOpenAi= getpass.getpass("Introduce API KEY sin comillas: ")

KeyboardInterrupt: Interrupted by user

In [ ]:
#automatico

from google.colab import userdata
keyOpenAi = userdata.get('KeyOpenAi')

In [ ]:
!pip install openai

import os
from openai import OpenAI

# Inicialización del cliente con la API moderna
client = OpenAI(api_key=keyOpenAi)

# Selección del modelo precio  Input	Cached input	Output  https://platform.openai.com/docs/pricing
# ModelUsed="gpt-4.1-2025-04-14"        #$2.00 $0.50  S8.00
ModelUsed="gpt-4o-mini-2024-07-18"  #$0.15 $0.075 $0.60
# ModelUsed="o4-mini-2025-04-16"      #$1.10 $0.275 $4.40



In [ ]:
# código refacturizado: para hacer esto
#  extrayendo el texto de una fila de un dataframe,
#   usándolo como entrada para la API de ChatCompletion de OpenAI,
#   e insertando la respuesta en otro dataframe.
# Este código asume que tienes una cierta flexibilidad en la elección de tus filas de inicio y fin.
#  Si siempre estás procesando la siguiente fila y no tienes un rango fijo,
#   podrías considerar almacenar la "fila actual" en una variable persistente o en un archivo, y actualizarla cada vez que ejecutes la función.

# La función generate_response_and_update_df toma el DataFrame donde quieres guardar la respuesta,
#  el índice de la fila a actualizar, el modelo de OpenAI a utilizar, los mensajes del sistema y del usuario, y la temperatura para la generación de texto.
#  Luego realiza la llamada a la API de OpenAI, imprime la información de uso y el contenido de la respuesta, actualiza el DataFrame con la información de la respuesta y finalmente devuelve el DataFrame actualizado.
import time

def generate_response_and_update_df(df, row, model, system_prompt, user_prompt, temperature):
    # Llamada a la API de OpenAI con la sintaxis actualizada
    response = client.chat.completions.create(
      model=model,
      messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
      temperature=temperature
    )

    # Imprimir la información de los tokens y el contenido de respuesta
    print(response.usage)
    print(response.choices[0].message.content)

    # Añadir la respuesta GPT al DataFrame
    df.loc[row, 'GPTId'] = row
    df.loc[row, 'GPTContent'] = response.choices[0].message.content
    df.loc[row, 'GPTResponse'] = str(response)
    df.loc[row, 'GPTUsage'] = str(response.usage)
    df.loc[row, 'PromtAgregado'] = user_prompt

    return df

# Llamada a la función en un bucle
start_row = 0
end_row = 50
fixed_delay = 0.05  # la idea es no pasarse de 200 peticiones por minuto, incluso sin delay va a unas 30 peticiones por minuto

for row in range(start_row, end_row):
    PromtUser1 = DataPapers['Prompt'].iloc[row]
    df3 = generate_response_and_update_df(df3, row, ModelUsed, PromtSystemB, PromtUser1, Temperatura)
    print(f"Procesada fila {row + 1} de {end_row}")

    # Guardar periódicamente para no perder progreso
    if row % 50 == 0 and row > 0:  # Cada 50 filas
        df3.to_csv(f"resultados_parciales_hasta_fila_{row}.csv", index=False)
        print(f"Progreso guardado hasta la fila {row}")

    # Pausa fija entre peticiones
    if row < end_row - 1:  # No esperar después de la última fila
        time.sleep(fixed_delay)

# Guardar resultado final
df3.to_csv("resultados_finales.csv", index=False)
print("Proceso completado!")

df3





CompletionUsage(completion_tokens=86, prompt_tokens=734, total_tokens=820, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
##id0001#Students' evaluation of teaching; assessment indicators; teaching quality; course evaluation; optimization; reasonable results; normal distribution; teaching effect; main problem of teaching#@Cat3Maybe@#The abstract discusses students' evaluation of teaching and the optimization of assessment indicators, but it does not explicitly mention the use of open-ended questions in the evaluation process, leaving some ambiguity regarding its alignment with the research focus.##
Procesada fila 1 de 50
CompletionUsage(completion_tokens=65, prompt_tokens=682, total_tokens=747, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejec

IndexError: single positional indexer is out-of-bounds

# Codigo especifico para GEMINI/Google API

In [4]:

# Para Gemini
!pip install google-generativeai

import time
import google.generativeai as genai
from google.colab import userdata

# Configurar la API de Gemini
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

def generate_response_and_update_df_gemini(df, row, model, system_prompt, user_prompt, temperature):
    # Configurar el modelo de Gemini
    model_instance = genai.GenerativeModel(model)

    # Combinar system prompt y user prompt para Gemini
    combined_prompt = f"Instrucciones del sistema: {system_prompt}\n\nConsulta del usuario: {user_prompt}"

    # Configurar parámetros de generación
    generation_config = genai.types.GenerationConfig(
        temperature=temperature,
        max_output_tokens=2048,
    )

    try:
        # Llamada a la API de Gemini
        response = model_instance.generate_content(
            combined_prompt,
            generation_config=generation_config
        )

        # Extraer información de la respuesta
        content = response.text

        # Imprimir información (Gemini no proporciona usage como OpenAI)
        print(f"Respuesta generada para fila {row}")
        print(content)

        # Añadir la respuesta al DataFrame
        df.loc[row, 'GPTId'] = row
        df.loc[row, 'GPTContent'] = content
        df.loc[row, 'GPTResponse'] = str(response)
        df.loc[row, 'GPTUsage'] = "No disponible en Gemini"
        df.loc[row, 'PromtAgregado'] = user_prompt

    except Exception as e:
        print(f"Error en fila {row}: {e}")
        df.loc[row, 'GPTId'] = row
        df.loc[row, 'GPTContent'] = f"Error: {e}"
        df.loc[row, 'GPTResponse'] = "Error"
        df.loc[row, 'GPTUsage'] = "Error"
        df.loc[row, 'PromtAgregado'] = user_prompt

    return df

# Llamada a la función en un bucle para Gemini
start_row = 0
end_row = 193
fixed_delay = 5  # Gemini tiene límites diferentes, ajusta según necesidad

# Especifica el modelo de Gemini que quieres usar ver https://ai.google.dev/gemini-api/docs/models?hl=es-419  limites de cada modelo https://ai.google.dev/gemini-api/docs/rate-limits?hl=es-419
ModelUsed = "gemini-2.5-flash-lite-preview-06-17"  # o "gemini-2.5-pro" "gemini-2.5-flash-lite-preview-06-17" "gemini-2.0-flash-lite"

for row in range(start_row, end_row):
    PromtUser1 = DataPapers['Prompt'].iloc[row]
    df3 = generate_response_and_update_df_gemini(df3, row, ModelUsed, PromtSystemB, PromtUser1, Temperatura)
    print(f"Procesada fila {row + 1} de {end_row}")

    # Guardar periódicamente para no perder progreso
    if row % 50 == 0 and row > 0:
        df3.to_csv(f"resultados_parciales_gemini_hasta_fila_{row}.csv", index=False)
        print(f"Progreso guardado hasta la fila {row}")

    # Pausa fija entre peticiones
    if row < end_row - 1:
        time.sleep(fixed_delay)

# Guardar resultado final
df3.to_csv("resultados_finales_gemini.csv", index=False)
print("Proceso completado con Gemini!")

Respuesta generada para fila 0
##id0001_0#Students' evaluation of teaching; assessment indicators; teaching quality; course evaluation; teaching effect#@Cat4Discard@#The abstract focuses on the methodology and results of student evaluations of teaching in terms of assessment indicators and their impact on course evaluation outcomes. It does not discuss the psychological health or well-being of university professors in relation to open-ended comments.###
Procesada fila 1 de 193
Respuesta generada para fila 1
##id0012_0#Flipped pedagogy; student evaluations of teaching; faculty concerns; course design; pedagogical techniques; traditional lecture; recorded lectures; flipped classroom model; student ratings; instructor perception; student liking; teaching evaluations; student perceptions; class format#@Cat4Discard@#While the abstract mentions student evaluations of teaching and faculty concerns, it focuses on the impact of pedagogical techniques on these evaluations and student perceptions

# Codigo especifico para ANTHROPIC/Claude API

In [ ]:
# Para Claude
!pip install anthropic

import time
from anthropic import Anthropic
from google.colab import userdata

# Configurar el cliente de Claude
client_claude = Anthropic(api_key=userdata.get('ClaudeAPIkei'))

def generate_response_and_update_df_claude(df, row, model, system_prompt, user_prompt, temperature):
    try:
        # Llamada a la API de Claude
        response = client_claude.messages.create(
            model=model,
            max_tokens=2048,
            temperature=temperature,
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_prompt}
            ]
        )

        # Extraer información de la respuesta
        content = response.content[0].text
        usage_info = f"Input tokens: {response.usage.input_tokens}, Output tokens: {response.usage.output_tokens}"

        # Imprimir información
        print(f"Tokens usados: {usage_info}")
        print(content)

        # Añadir la respuesta al DataFrame
        df.loc[row, 'GPTId'] = row
        df.loc[row, 'GPTContent'] = content
        df.loc[row, 'GPTResponse'] = str(response)
        df.loc[row, 'GPTUsage'] = usage_info
        df.loc[row, 'PromtAgregado'] = user_prompt

    except Exception as e:
        print(f"Error en fila {row}: {e}")
        df.loc[row, 'GPTId'] = row
        df.loc[row, 'GPTContent'] = f"Error: {e}"
        df.loc[row, 'GPTResponse'] = "Error"
        df.loc[row, 'GPTUsage'] = "Error"
        df.loc[row, 'PromtAgregado'] = user_prompt

    return df

# Llamada a la función en un bucle para Claude
start_row = 0
end_row = 50
fixed_delay = 12  # Claude tiene límites de rate, ajusta según tu plan

# Especifica el modelo de Claude que quieres usar ver info en https://docs.anthropic.com/en/docs/about-claude/models/overview
ModelUsed = "claude-sonnet-4-20250514"  # o "claude-opus-4-20250514" "claude-3-5-haiku-20241022"

for row in range(start_row, end_row):
    PromtUser1 = DataPapers['Prompt'].iloc[row]
    df3 = generate_response_and_update_df_claude(df3, row, ModelUsed, PromtSystemB, PromtUser1, Temperatura)
    print(f"Procesada fila {row + 1} de {end_row}")

    # Guardar periódicamente para no perder progreso
    if row % 50 == 0 and row > 0:
        df3.to_csv(f"resultados_parciales_claude_hasta_fila_{row}.csv", index=False)
        print(f"Progreso guardado hasta la fila {row}")

    # Pausa fija entre peticiones
    if row < end_row - 1:
        time.sleep(fixed_delay)

# Guardar resultado final
df3.to_csv("resultados_finales_claude.csv", index=False)
print("Proceso completado con Claude!")

Tokens usados: Input tokens: 715, Output tokens: 94
##id0001#Students' evaluation of teaching; assessment indicators; course evaluation; teaching quality assessment; normal distribution; teaching effect evaluation#@Cat4Discard@#This study focuses on quantitative assessment indicators and statistical distribution of evaluation results rather than open-ended student feedback or psychological impact on faculty. It examines optimization of assessment systems but does not address open-ended comments or their effects on professors' mental health and wellbeing##
Procesada fila 1 de 50
Tokens usados: Input tokens: 673, Output tokens: 91
##id0002#Online Student Evaluation of Teaching System; implementation process; stakeholder consultation; user training; primary data accuracy; system productivity enhancement#@Cat1InsufficInformat@#The abstract focuses on the technical implementation and operational aspects of an online student evaluation system but does not specify whether the evaluation inclu

# codigo comun para guardar los resultados

In [5]:
# Guardamos el DataFrame en un archivo Excel

# Definimos la ruta y el nombre del archivo incluyendo la fecha actual
from datetime import datetime
fecha_actual = datetime.now().strftime("%Y-%m-%d %H:%M")
nombre_base = f"{ModelUsed} B2-{Temperatura}"
nombre_fichero = f"{nombre_base}_{fecha_actual}.xlsx"

df3.to_excel(nombre_fichero, index=False)

# mejoras

